In [1]:
import pandas as pd
import os

# Directory containing the CSV files
directory = 'blackrock/'

# List to hold DataFrames
dataframes = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        # Read each CSV file into a DataFrame
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_output.csv', index=False)

print('All CSV files have been merged successfully.')


All CSV files have been merged successfully.


In [6]:
def filter_transactions(df, from_label_keyword, to_label_keyword, tx_type='inflow'):
    # Drop duplicates based on transactionHash to get unique values
    df_unique = df.drop_duplicates(subset='transactionHash')
    
    # Filter by type
    df_filtered_type = df_unique[df_unique['type'] == tx_type]
    
    # Filter rows where fromLabel contains the specified keyword
    df_filtered_from = df_filtered_type[df_filtered_type['fromLabel'].str.contains(from_label_keyword, case=False, na=False)]
    
    # Filter rows where toLabel contains the specified keyword
    df_filtered_to = df_filtered_from[df_filtered_from['toLabel'].str.contains(to_label_keyword, case=False, na=False)]
    
    return df_filtered_to


# Apply the filter function
filtered_df = filter_transactions(merged_df, 'Coinbase', 'Blackrock')

print(len(merged_df))
# Display the filtered DataFrame
print(len(filtered_df))

416
103


In [7]:
filtered_df.head()

,transactionHash,fromAddress,fromLabel,fromIsContract,toAddress,toLabel,toIsContract,tokenAddress,type,blockTimestamp,blockNumber,blockHash,tokenName,tokenSymbol,tokenDecimals,unitValue,tokenId,historicalUSD,chain
0,4f3b841eb7c598886f82f0563946e66bdbbe39ff3b9f1d...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,Coinbase Prime: Hot Wallet (3MqUP),False,bc1qsmrl3ckx45qc3c46q7gf3h752wtjyatnrql4tr,BlackRock: IBIT Bitcoin ETF (bc1qs),False,NaN,inflow,2024-07-17T17:36:06Z,852640,00000000000000000001166e09db910812c09e6a8e3de0...,Bitcoin,BTC,8,600.0,bitcoin,38593200.0,bitcoin
1,7e25accfd53f935b1b58c2fb9095852e9f15e552f8f18b...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,Coinbase Prime: Hot Wallet (3MqUP),False,bc1quhdw4pumtqth860casf29gfkayj8q7xwpa2akt,BlackRock: IBIT Bitcoin ETF (bc1qu),False,NaN,inflow,2024-07-17T17:36:06Z,852640,00000000000000000001166e09db910812c09e6a8e3de0...,Bitcoin,BTC,8,600.0,bitcoin,38593200.0,bitcoin
2,5fc39396a697c167ce1028ad477a32dba2cb5ce4f72486...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,Coinbase Prime: Hot Wallet (3MqUP),False,bc1qa75ery0z56kxzrn6swp3ntfvenqfuq8as2ss2e,BlackRock: IBIT Bitcoin ETF (bc1qa),False,NaN,inflow,2024-07-17T17:36:06Z,852640,00000000000000000001166e09db910812c09e6a8e3de0...,Bitcoin,BTC,8,74.0,bitcoin,4759828.0,bitcoin
3,db5fe3b61d7e63e9d72c99f0b6705b4002ef93141307be...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,Coinbase Prime: Hot Wallet (3MqUP),False,bc1q5zt0nqd7f7t7ra3j48tjspa5ucn0gwff7dvwa0,BlackRock: IBIT Bitcoin ETF (bc1q5),False,NaN,inflow,2024-07-17T17:36:06Z,852640,00000000000000000001166e09db910812c09e6a8e3de0...,Bitcoin,BTC,8,600.0,bitcoin,38593200.0,bitcoin
4,4ca27cc8ba0ecbc227fc2c348ffa7a26395a1ce2b3369a...,3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd,Coinbase Prime: Hot Wallet (3MqUP),False,bc1qptj7s7rwthdravz2uv730gns7mragcxtz8re8f,BlackRock: IBIT Bitcoin ETF (bc1qp),False,NaN,inflow,2024-07-17T17:16:38Z,852637,00000000000000000001cd74cbf9c942ed0949a3209738...,Bitcoin,BTC,8,600.0,bitcoin,38472600.0,bitcoin


In [8]:
def calculate_average_price(df, token_id):
    # Filter the DataFrame based on the tokenId
    df_filtered = df[df['tokenId'] == token_id]
    
    # Check if there are rows after filtering
    if df_filtered.empty:
        return None
    
    # Calculate the weighted average price
    df_filtered['total_value'] = df_filtered['unitValue'] * df_filtered['historicalUSD']
    total_unit_value = df_filtered['unitValue'].sum()
    
    if total_unit_value == 0:
        return None
    
    weighted_average_price = df_filtered['total_value'].sum() / total_unit_value
    
    return weighted_average_price

# Calculate the average price for the specified tokenId
average_price = calculate_average_price(filtered_df, 'bitcoin')

# Display the average price
if average_price is not None:
    print(f'The average price for bitcoin is: {average_price}')
else:
    print('No data available for the specified tokenId or unitValue is zero.')


The average price for bitcoin is: 36786887.6993427
